In [ ]:
import os
import requests
import json
from openai import OpenAI
import httpx

httpx_client = httpx.Client(http2=True, verify=False)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]


openai = OpenAI(base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=httpx_client)

system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a joke for an audience of Cloud Foundry enthusiats"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

completion = openai.chat.completions.create(model=chat_credentials["model_name"], messages=prompts)
print(completion.choices[0].message.content)

Add 'matplotlib' dependency using the Terminal and 'uv pip install matplotlib' command

In [ ]:
import matplotlib.pyplot as plt
import json
import random

# Ask AI for data in a specific format
response = openai.chat.completions.create(
    model=chat_credentials["model_name"],
    messages=[{"role": "user", "content": """Give me 5 fictional Cloud Foundry application names with their memory usage in MB. 
    Format your response exactly like this JSON:
    {"applications": [{"name": "AppName", "memory_mb": 512, "instances": 3, "space": "development"}]}
    
    Make the app names realistic like microservices (user-service, payment-api, etc.) and memory between 256-2048 MB."""}]
)

print("AI Response:", response.choices[0].message.content)

try:
    # Try to parse as JSON
    ai_text = response.choices[0].message.content
    
    # Extract JSON from response (in case there's extra text)
    start = ai_text.find('{')
    end = ai_text.rfind('}') + 1
    json_str = ai_text[start:end]
    
    data = json.loads(json_str)
    app_names = [app["name"] for app in data["applications"]]
    memory_usage = [app["memory_mb"] for app in data["applications"]]
    instances = [app.get("instances", random.randint(1, 5)) for app in data["applications"]]
    spaces = [app.get("space", "production") for app in data["applications"]]
    
except (json.JSONDecodeError, KeyError, ValueError) as e:
    print(f"JSON parsing failed: {e}")
    print("Using fallback Cloud Foundry data")
    app_names = ['user-service', 'payment-api', 'notification-worker', 'auth-gateway', 'analytics-engine']
    memory_usage = [random.randint(256, 2048) for _ in app_names]
    instances = [random.randint(1, 5) for _ in app_names]
    spaces = ['production', 'staging', 'development', 'production', 'staging']

print(f"Applications: {app_names}")
print(f"Memory Usage (MB): {memory_usage}")
print(f"Instances: {instances}")
print(f"Spaces: {spaces}")

# Create the main chart - Memory Usage
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Color mapping for Cloud Foundry spaces
space_colors = {
    'production': '#e74c3c',    # Red
    'staging': '#f39c12',       # Orange  
    'development': '#27ae60',   # Green
    'testing': '#3498db'        # Blue
}

colors = [space_colors.get(space, '#95a5a6') for space in spaces]

# Chart 1: Memory Usage by Application
bars1 = ax1.bar(app_names, memory_usage, color=colors, alpha=0.8, edgecolor='white', linewidth=2)
ax1.set_title('Cloud Foundry Applications - Memory Allocation', fontsize=16, fontweight='bold', pad=20)
ax1.set_ylabel('Memory Usage (MB)', fontsize=12)
ax1.set_xlabel('Applications', fontsize=12)

# Rotate x-axis labels for better readability
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')

# Add value labels on bars with instance count
for bar, memory, instance_count, space in zip(bars1, memory_usage, instances, spaces):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + max(memory_usage)*0.01, 
             f'{memory} MB\n({instance_count} instances)', 
             ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Add space label at the bottom of each bar
    ax1.text(bar.get_x() + bar.get_width()/2, height * 0.1, 
             space.upper(), 
             ha='center', va='bottom', fontsize=8, color='white', fontweight='bold')

# Add grid for better readability
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_axisbelow(True)

# Chart 2: Instance Count by Application
bars2 = ax2.bar(app_names, instances, color=colors, alpha=0.8, edgecolor='white', linewidth=2)
ax2.set_title('Cloud Foundry Applications - Instance Count', fontsize=16, fontweight='bold', pad=20)
ax2.set_ylabel('Number of Instances', fontsize=12)
ax2.set_xlabel('Applications', fontsize=12)

# Rotate x-axis labels for better readability
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

# Add value labels on bars
for bar, instance_count in zip(bars2, instances):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height + 0.05, 
             str(instance_count), 
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add grid for better readability
ax2.grid(axis='y', alpha=0.3, linestyle='--')
ax2.set_axisbelow(True)
ax2.set_ylim(0, max(instances) + 1)

# Create custom legend for spaces
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=space.title()) 
                  for space, color in space_colors.items() if space in spaces]

ax1.legend(handles=legend_elements, title='CF Spaces', loc='upper right', 
          bbox_to_anchor=(1, 1), framealpha=0.9)

# Add summary statistics text box
total_memory = sum(memory_usage)
total_instances = sum(instances)
avg_memory = total_memory / len(app_names)

stats_text = f"""Cloud Foundry Summary:
• Total Memory: {total_memory:,} MB
• Total Instances: {total_instances}
• Average Memory/App: {avg_memory:.0f} MB
• Applications: {len(app_names)}"""

# Add text box with statistics
ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

plt.tight_layout()
plt.show()

# Additional Cloud Foundry specific visualization
print("\n" + "="*50)
print("CLOUD FOUNDRY DEPLOYMENT SUMMARY")
print("="*50)

for i, (app, memory, inst, space) in enumerate(zip(app_names, memory_usage, instances, spaces), 1):
    status = "🟢 RUNNING" if memory < 1000 else "🟡 HIGH MEMORY"
    print(f"{i}. {app.upper()}")
    print(f"   └─ Space: {space}")
    print(f"   └─ Memory: {memory} MB x {inst} instances = {memory * inst} MB total")
    print(f"   └─ Status: {status}")
    print()

# Create a Cloud Foundry org/space hierarchy visualization
print("CF ORG/SPACE HIERARCHY:")
print("└─ my-org")
for space in set(spaces):
    apps_in_space = [app for app, s in zip(app_names, spaces) if s == space]
    print(f"   ├─ {space}")
    for app in apps_in_space:
        print(f"   │  └─ {app}")
    print()

Add 'gradio' dependency using the Terminal and 'uv pip install gradio' command

In [ ]:
import gradio as gr

def chat_with_ai(message, history):
    """
    Function to handle chat interactions with the AI model
    """
    try:
        # Prepare the conversation history for the API
        messages = [{"role": "system", "content": "You are an assistant that is great at telling jokes. Always respond directly with the joke or answer. Do not show your thinking process or use thinking tags. Just give the final response immediately."}]
        
        # Add conversation history (now in messages format)
        for msg in history:
            messages.append(msg)
        
        # Add the current message
        messages.append({"role": "user", "content": message})
        
        # Make API call with streaming enabled and higher token limit
        completion = openai.chat.completions.create(
            model=chat_credentials["model_name"], 
            messages=messages,
            max_tokens=800,  # Increased token limit to avoid cutoffs
            temperature=0.7,
            stream=True
        )
        
        # Collect the complete streamed response with timeout handling
        full_response = ""
        print("=== STREAMING RESPONSE ===")
        
        try:
            for chunk in completion:
                if chunk.choices[0].delta.content:
                    content = chunk.choices[0].delta.content
                    full_response += content
                    print(content, end='', flush=True)
                
                # Check if we have a complete response (ends with </think> followed by content)
                if '</think>' in full_response:
                    # Continue streaming to get content after </think>
                    continue
                elif full_response.strip().endswith('</think>'):
                    # Stream ended right at </think>, this might be incomplete
                    print("\n[WARNING: Stream ended at </think> - might be incomplete]")
                    break
        except Exception as e:
            print(f"\n[STREAMING ERROR: {e}]")
        
        print("\n=== STREAMING COMPLETE ===")
        response = full_response
        print(f"=== FULL API RESPONSE START ===")
        print(response)
        print(f"=== FULL API RESPONSE END ===")
        print(f"Response length: {len(response) if response else 0}")
        print(f"Response type: {type(response)}")
        
        # Clean the response - simply extract what comes after </think>
        cleaned_response = ""
        
        if response and '</think>' in response:
            # Find the last </think> tag and get everything after it
            think_end_index = response.rfind('</think>')
            after_think = response[think_end_index + len('</think>'):].strip()
            cleaned_response = after_think
            print(f"Content after </think>: '{cleaned_response}'")
 
        print(f"Cleaned Response: {cleaned_response}")  # Debug print
        return cleaned_response
        
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(f"Error occurred: {error_msg}")  # Debug print
        return error_msg

# Create Gradio interface
with gr.Blocks(title="AI Joke Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 AI Joke Assistant")
    gr.Markdown("Chat with an AI that's great at telling jokes! Ask for jokes about any topic.")
    
    chatbot = gr.Chatbot(
        height=400,
        placeholder="Start chatting with your AI joke assistant...",
        show_copy_button=True,
        type="messages"  # Use the new messages format
    )
    
    with gr.Row():
        msg = gr.Textbox(
            placeholder="Ask for a joke or chat with the AI...",
            show_label=False,
            scale=4
        )
        send_btn = gr.Button("Send", variant="primary", scale=1)
    
    with gr.Row():
        clear_btn = gr.Button("Clear Chat", variant="secondary")
    
    # Handle sending messages
    def respond(message, chat_history):
        print(f"User message: {message}")  # Debug print
        print(f"Current history length: {len(chat_history)}")  # Debug print
        
        if not message.strip():
            return "", chat_history
        
        bot_message = chat_with_ai(message, chat_history)
        print(f"Bot response: {bot_message}")  # Debug print
        
        # Add user message and bot response to history (using messages format)
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": bot_message})
        
        print(f"Updated history length: {len(chat_history)}")  # Debug print
        return "", chat_history
    
    # Event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(lambda: [], None, chatbot, queue=False)
    
    # Example prompts
    gr.Examples(
        examples=[
            "Tell a joke for Cloud Foundry enthusiasts",
            "Give me a programming joke",
            "Tell me a dad joke",
            "What's a funny joke about containers?",
            "Make me laugh with a tech joke"
        ],
        inputs=msg
    )

# Launch the interface
demo.launch(
    share=True,  # Creates a public link
    server_name="0.0.0.0",  # Allows access from any IP
    server_port=7860,  # Default Gradio port
    show_error=True,
    quiet=False
)